# Agentic Search

## First: Example without using the client. Minimun we need three steps.
1- Create a search function using ducckduckgo_search that will return urls.

2- Scrape info using the library Beautiful Soup.

***Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with your favorite parser to provide idiomatic ways of navigating, searching, and modifying the parse tree.***

3- Print the info

In [1]:
# Load all libraries
from dotenv import load_dotenv
import os

# load environment variables from .env file
_ = load_dotenv()

In [4]:
import requests
from bs4 import BeautifulSoup
import re
from duckduckgo_search import DDGS

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://en.wikipedia.org/wiki/Paraguay",
            "https://en.wikipedia.org/wiki/Asunción"
            
        ]
        return results  

In [5]:
city = "Asunción"  # change to your city of interest

query = f"""
    where is the city of {city}?
    Which languages are spoken there?
    "en.wikipedia.org"
"""

for i in search(query):
    print(i)

returning previous results due to exception reaching ddg.
https://en.wikipedia.org/wiki/Paraguay
https://en.wikipedia.org/wiki/Asunción


In [6]:
# Function to scrape weather information from a given URL
def scrape_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:10000]) # limit long outputs

returning previous results due to exception reaching ddg.
Website: https://en.wikipedia.org/wiki/Paraguay


<body class="skin--responsive skin-vector skin-vector-search-vue mediawiki ltr sitedir-ltr mw-hide-empty-elt ns-0 ns-subject mw-editable page-Paraguay rootpage-Paraguay skin-vector-2022 action-view"><a class="mw-jump-link" href="#bodyContent">Jump to content</a>
<div class="vector-header-container">
<header class="vector-header mw-header no-font-mode-scale">
<div class="vector-header-start">
<nav aria-label="Site" class="vector-main-menu-landmark">
<div class="vector-dropdown vector-main-menu-dropdown vector-button-flush-left vector-button-flush-right" id="vector-main-menu-dropdown" title="Main menu">
<input aria-haspopup="true" aria-label="Main menu" class="vector-dropdown-checkbox" data-event-name="ui.dropdown-vector-main-menu-dropdown" id="vector-main-menu-dropdown-checkbox" role="button" type="checkbox"/>
<label aria-hidden="true" class="vector-dropdown-label cdx-button cdx-b

In [7]:
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    if tag.text:
        text = tag.text.strip()
        if re.search(r'\b(?:temperature|weather|forecast|humidity|wind)\b', text, re.IGNORECASE):
            weather_data.append(text)

if weather_data:
    print("Weather Information:")
    for data in weather_data:
        print(f"- {data}")

Weather Information:
- The overall climate is tropical to subtropical. Like most lands in the region, Paraguay has only wet and dry periods. Winds play a major role in influencing Paraguay's weather: between October and March, warm winds blow from the Amazon Basin in the north, while the period between May and August brings cold winds from the Andes.
- The absence of mountain ranges to provide a natural barrier allows winds to develop speeds as high as 161 km/h (100 mph). This also leads to significant changes in temperature within a short span of time; between April and September, temperatures will sometimes drop below freezing. January is the hottest summer month, with an average daily temperature of 28.9 degrees Celsius (84 degrees F).
- Paraguay's most important urban areas are located along the Argentina-Paraguay border: Asunción, Alberdi, Encarnación, Pilar and Ciudad del Este, the latter being the third most important free commercial zone in the world, only trailing behind Miami

In [16]:
city_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.text.strip()
    city_data.append(text)

city_data = "\n".join(city_data)

city_data = re.sub(r'\s+', ' ', city_data)  # remove extra whitespace

print(f"Website: {url}\n\n")
print(city_data)  # limit long outputs

Website: https://en.wikipedia.org/wiki/Paraguay


Contents Paraguay in South America (gray) Paraguay,[c] officially the Republic of Paraguay,[d] is a landlocked American country located in the central region of South America. It is a unitary state with a territory composed of a capital district and seventeen provinces. Its capital and largest city is Asunción. Paraguay is a presidential republic and a state governed by the rule of law. It is a founding member of Mercosur, along with Argentina, Brazil and Uruguay. Spanish conquistadores arrived in 1524, and in 1537 established the city of Asunción, the first capital of the Governorate of the Río de la Plata.[9] During the 17th century, Paraguay was the center of Jesuit missions, where the native Guaraní people were converted to Christianity and introduced to European culture.[10] After the expulsion of the Jesuits from Spanish territories in 1767, Paraguay increasingly became a peripheral colony. Following independence from Spain in the

## Second: Example using the client. Simple output

In [17]:
from tavily import TavilyClient
# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

print(f"Running query: {query} with Tavily search...")
result = client.search(query, max_results=1)
result

Running query: 
    where is the city of Asunción?
    Which languages are spoken there?
    "en.wikipedia.org"
 with Tavily search...


{'query': '\n    where is the city of Asunción?\n    Which languages are spoken there?\n    "en.wikipedia.org"\n',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Asunci%C3%B3n',
   'title': 'Asunción - Wikipedia',
   'content': 'The metropolitan area, called Gran Asunción, includes the cities of San Lorenzo "San Lorenzo (Paraguay)"), Fernando de la Mora "Fernando de la Mora (Paraguay)"), Lambaré, Luque, Mariano Roque Alonso, Ñemby, San Antonio, Limpio, Capiatá and Villa Elisa "Villa Elisa (Paraguay)"), which are part of the Central Department. Asunción has been the first completely urban city in Paraguay since approximately the middle of the 20th century. Because the Paraguay River runs right next to Asunción the city is served by a river terminal in the downtown area. Being the third-oldest capital city of South America, after Quito and Lima, Asunción has plenty to offer, culturally speaking, from Spanish colonial-era 

### See in a JSON format

In [20]:
import json
from pygments import highlight, lexers, formatters
# print the first result
if result and len(result) > 0:
    data = result["results"][0]

    if type(result) is dict and "results" in result:
        # pretty print JSON with syntax highlighting
        formatted_json = json.dumps(data, indent=4)

    if type(result) is str:
        data = data.replace("'", '"') # replace single quotes with double quotes for valid JSON format
        parsed_json = json.loads(data)
        formatted_json = json.dumps(parsed_json, indent=4)

# formatted_json

colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)

{
    "url": "https://en.wikipedia.org/wiki/Asunci%C3%B3n",
    "title": "Asunci\u00f3n - Wikipedia",
    "content": "The metropolitan area, called Gran Asunci\u00f3n, includes the cities of San Lorenzo \"San Lorenzo (Paraguay)\"), Fernando de la Mora \"Fernando de la Mora (Paraguay)\"), Lambar\u00e9, Luque, Mariano Roque Alonso, \u00d1emby, San Antonio, Limpio, Capiat\u00e1 and Villa Elisa \"Villa Elisa (Paraguay)\"), which are part of the Central Department. Asunci\u00f3n has been the first completely urban city in Paraguay since approximately the middle of the 20th century. Because the Paraguay River runs right next to Asunci\u00f3n the city is served by a river terminal in the downtown area. Being the third-oldest capital city of South America, after Quito and Lima, Asunci\u00f3n has plenty to offer, culturally speaking, from Spanish colonial-era buildings (Baroque to neo-Gothic), museums or urban parks, this classic city also hosts several symphony orchestras, ballet, opera and th

In [23]:
# Just for fun, let's run a search with an answer and dates of the sources
# run search
result = client.search("What were the advances in AI at the beginning of 2025? Include answer and dates of your sources",
                       include_answer=True)

# print the answer
result["answer"]

'In early 2025, AI advancements focused on generalist AI agents, multimodal models, and significant investments in AI infrastructure. Key highlights included the launch of advanced AI assistants and substantial funding for AI projects.'